In [1]:
from datasets import load_dataset
import numpy as np
from loguru import logger

# Load data

In [2]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "5core_timestamp_Books", trust_remote_code=True)

In [3]:
dataset['train']

Dataset({
    features: ['user_id', 'parent_asin', 'rating', 'timestamp'],
    num_rows: 8733855
})

In [4]:
dataset['valid']

Dataset({
    features: ['user_id', 'parent_asin', 'rating', 'timestamp'],
    num_rows: 426209
})

In [5]:
def parse_dtype(df):
    return (
        df
        .assign(
            rating=lambda df: df['rating'].astype(float),
            timestamp=lambda df: df['timestamp'].astype(int)
        )
    )

train_raw = dataset['train'].to_pandas().pipe(parse_dtype)
val_raw = dataset['valid'].to_pandas().pipe(parse_dtype)

# Sample data

In [6]:
SAMPLE_TRAIN_USERS = 30000
if SAMPLE_TRAIN_USERS:
    random_seed = 42
    np.random.seed(random_seed)
    
    # Get users present in both train and val datasets
    users_in_train = train_raw['user_id'].unique()
    users_in_val = val_raw['user_id'].unique()
    common_users = np.intersect1d(users_in_val, users_in_train)

    # Sample users from the common users
    sample_users = np.random.choice(common_users, size=SAMPLE_TRAIN_USERS, replace=False)
    
    # Fetch all interactions of the sampled users in both datasets
    val_sample = val_raw[val_raw['user_id'].isin(sample_users)]
    train_sample = train_raw[train_raw['user_id'].isin(sample_users)]

    # Ensure all items in val_sample exist in train_sample
    train_items = train_sample['parent_asin'].unique()
    val_sample = val_sample[val_sample['parent_asin'].isin(train_items)]

    # Update item and user lists after filtering
    val_items = val_sample['parent_asin'].unique()
    train_users = train_sample['user_id'].unique()
    val_users = val_sample['user_id'].unique()

    # Logging
    logger.info(f"{len(train_items)=}, {len(train_users)=}")
    logger.info(f"{len(val_items)=}, {len(val_users)=}")
    val_users_in_train = set(val_users).intersection(set(train_users))
    val_items_in_train = set(val_items).intersection(set(train_items))
    logger.info(f"Percentage of val users in train: {len(val_users_in_train) / len(val_users):,.0%}")
    logger.info(f"Percentage of val items in train: {len(val_items_in_train) / len(val_items):,.0%}")
    logger.info(f"Sparsity: {1 - len(train_sample) / (len(train_items) * len(train_users)):,.4%}")

2024-09-20 13:22:50.904 | INFO     | __main__:<module>:28 - len(train_items)=195057, len(train_users)=30000
2024-09-20 13:22:50.905 | INFO     | __main__:<module>:29 - len(val_items)=20039, len(val_users)=18397
2024-09-20 13:22:50.927 | INFO     | __main__:<module>:32 - Percentage of val users in train: 100%
2024-09-20 13:22:50.928 | INFO     | __main__:<module>:33 - Percentage of val items in train: 100%
2024-09-20 13:22:50.928 | INFO     | __main__:<module>:34 - Sparsity: 99.9922%


In [7]:
train_sample

,user_id,parent_asin,rating,timestamp
23,AHXBL3QDWZGJYH7A5CMPFNUPMF7Q,0920668372,5.0,1430056169000
24,AHXBL3QDWZGJYH7A5CMPFNUPMF7Q,1589255208,5.0,1443926150000
25,AHXBL3QDWZGJYH7A5CMPFNUPMF7Q,2764322836,5.0,1463967052000
26,AHXBL3QDWZGJYH7A5CMPFNUPMF7Q,2764330898,5.0,1489085694000
27,AHXBL3QDWZGJYH7A5CMPFNUPMF7Q,0062380761,5.0,1526591330983
...,...,...,...,...
8727810,AHCRTEBU4KY4M4FMHZKURTTZJVMA,1119742714,5.0,1596401789395
8727811,AHCRTEBU4KY4M4FMHZKURTTZJVMA,1645436284,5.0,1620769258437
8729931,AFSYIYI3FQPLRS3P7PTGN6H6FHGA,1940189004,5.0,1412824706000
8729932,AFSYIYI3FQPLRS3P7PTGN6H6FHGA,1939650380,5.0,1475293391000


In [8]:
val_sample

,user_id,parent_asin,rating,timestamp
4,AHXBL3QDWZGJYH7A5CMPFNUPMF7Q,0451450523,2.0,1635710722120
113,AGUWL2R2JFLC3K65HLD6AHJV3KBA,1439153663,5.0,1641321916399
135,AEIHN6JDLYL2QYCAOCBVEUPHU5EA,1680522876,5.0,1652558905635
136,AEIHN6JDLYL2QYCAOCBVEUPHU5EA,1450805752,5.0,1652559124250
144,AGBWZSPAIQMKAYQNMIUXJFRNRDXA,1441316825,5.0,1645646147624
...,...,...,...,...
425449,AGVLANGEASFXMR6VIQL5A7E7FARA,1401270727,1.0,1632619642935
425450,AGVLANGEASFXMR6VIQL5A7E7FARA,1779509510,5.0,1641951824343
425486,AFNXBSDR2QYHGIQ2GZK2WE7BZQIQ,1626347123,5.0,1654723852802
425660,AG2OONX4BPYUIAF4BY4LRVN4VCAA,1777488206,2.0,1646404539903


In [9]:
train_sample.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
user_id,456639,30000,AGRCXV4NM763VGZGD6BQ2VGNCJNA,1708,NaN,NaN,NaN,NaN,NaN,NaN,NaN
parent_asin,456639,195057,B00L9B7IKE,498,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rating,456639.0,NaN,NaN,NaN,4.341832,1.007625,1.0,4.0,5.0,5.0,5.0
timestamp,456639.0,NaN,NaN,NaN,1483004597672.758545,117134440629.050644,854697682000.0,1420163935500.0,1505882857714.0,1574227770225.5,1628643301627.0


# Persist sample

In [10]:
train_sample.to_parquet("../data/train.parquet")
val_sample.to_parquet("../data/val.parquet")

# Archive